In [1]:
import pandas as pd
import numpy as np

In [18]:
spike_clusters = pd.read_csv('../../data/covigator_clustering_results_20220606201844.csv')
spike_clusters['position'] = spike_clusters.variant_id.transform(lambda i: i.split(":")[0])
spike_clusters.head()

,cluster,cluster_jaccard_mean,hgvs_p,variant_id,position
0,0,0.972,p.L5F,21575:C>T,21575
1,0,0.972,p.N148T,22005:A>C,22005
2,0,0.972,p.D253G,22320:A>G,22320
3,0,0.972,p.D574Y,23282:G>T,23282
4,0,0.972,p.L176F,22088:C>T,22088


In [19]:
lineage_mutations = pd.read_csv('../../data/lineage_mutations.tsv', sep='\t')
# creates the HGVS p code for matching
lineage_mutations['hgvs_p'] = lineage_mutations.mutation.transform(lambda m: 'p.' + m)
lineage_mutations.head()

,sites,lineage,parent_lineage,protein,mutation,position,reference,alternate,protein_normalized,hgvs_p
0,nsp3:L741F,A.23.1,NaN,nsp3,L741F,741.0,L,F,ORF1ab,p.L741F
1,nsp6:M86I,A.23.1,NaN,nsp6,M86I,86.0,M,I,ORF1ab,p.M86I
2,nsp6:L98F,A.23.1,NaN,nsp6,L98F,98.0,L,F,ORF1ab,p.L98F
3,nsp6:M183I,A.23.1,NaN,nsp6,M183I,183.0,M,I,ORF1ab,p.M183I
4,S:R102I,A.23.1,NaN,S,R102I,102.0,R,I,S,p.R102I


In [20]:
# filter out synonymous mutations and keep only mutations in spike protein
spike_lineage_mutations = lineage_mutations[(lineage_mutations.protein != 'nuc') & (lineage_mutations.protein_normalized == 'S')]

In [21]:
spike_lineage_mutations

,sites,lineage,parent_lineage,protein,mutation,position,reference,alternate,protein_normalized,hgvs_p
4,S:R102I,A.23.1,NaN,S,R102I,102.0,R,I,S,p.R102I
5,S:F157L,A.23.1,NaN,S,F157L,157.0,F,L,S,p.F157L
6,S:V367F,A.23.1,NaN,S,V367F,367.0,V,F,S,p.V367F
7,S:Q613H,A.23.1,NaN,S,Q613H,613.0,Q,H,S,p.Q613H
8,S:P681R,A.23.1,NaN,S,P681R,681.0,P,R,S,p.P681R
...,...,...,...,...,...,...,...,...,...,...
546,S:G142D,AY.4.2,NaN,S,G142D,142.0,G,D,S,p.G142D
547,S:L452R,AY.4.2,NaN,S,L452R,452.0,L,R,S,p.L452R
548,S:T478K,AY.4.2,NaN,S,T478K,478.0,T,K,S,p.T478K
549,S:P681R,AY.4.2,NaN,S,P681R,681.0,P,R,S,p.P681R


In [22]:
spike_clusters_lineages = pd.merge(spike_clusters, spike_lineage_mutations[['hgvs_p', 'lineage']], 
                                   on='hgvs_p', how='left').sort_values(['cluster', 'position'])
spike_clusters_lineages

,cluster,cluster_jaccard_mean,hgvs_p,variant_id,position,lineage
0,0,0.972,p.L5F,21575:C>T,21575,B.1.526
1,0,0.972,p.N148T,22005:A>C,22005,NaN
4,0,0.972,p.L176F,22088:C>T,22088,NaN
2,0,0.972,p.D253G,22320:A>G,22320,B.1.526
3,0,0.972,p.D574Y,23282:G>T,23282,NaN
...,...,...,...,...,...,...
125,9,0.127,p.Y144del,21990:TTTA>T,21990,NaN
124,9,0.127,p.A570D,23271:C>A,23271,B.1.1.7
122,9,0.127,p.T716I,23709:C>T,23709,B.1.1.7
126,9,0.127,p.S982A,24506:T>G,24506,B.1.1.7


In [23]:
spike_clusters_lineages[~spike_clusters_lineages.lineage.isna()]

,cluster,cluster_jaccard_mean,hgvs_p,variant_id,position,lineage
0,0,0.972,p.L5F,21575:C>T,21575,B.1.526
2,0,0.972,p.D253G,22320:A>G,22320,B.1.526
26,1,0.424,p.T19R,21618:C>G,21618,B.1.617.2
27,1,0.424,p.T19R,21618:C>G,21618,B.1.617.3
28,1,0.424,p.T19R,21618:C>G,21618,AY.4
...,...,...,...,...,...,...
114,7,0.402,p.T859N,24138:C>A,24138,C.37
124,9,0.127,p.A570D,23271:C>A,23271,B.1.1.7
122,9,0.127,p.T716I,23709:C>T,23709,B.1.1.7
126,9,0.127,p.S982A,24506:T>G,24506,B.1.1.7


In [24]:
spike_clusters_lineages.to_csv('../../data/covigator_clustering_results_20220606201844_with_lineages.tsv', sep='\t', index=False)

In [25]:
spike_clusters_lineages['value'] = True
spike_clusters_lineages_pivot = spike_clusters_lineages[['cluster', 'hgvs_p', 'variant_id', 'position', 'lineage', 'value']].pivot(columns='lineage', index=['cluster', 'hgvs_p', 'variant_id', 'position'], values='value')\
.fillna('')\
.sort_values(['cluster', 'position'])
del spike_clusters_lineages_pivot[np.nan]

In [26]:
spike_clusters_lineages_pivot.reset_index(inplace=True)
spike_clusters_lineages_pivot.to_csv('../../data/covigator_clustering_results_20220606201844_with_lineages_pivot.tsv', sep='\t', index=False)